## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config_vacation_search import g_key
# print(g_key)

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Hambantota,LK,79.20,overcast clouds,6.1241,81.1185,Bungalow 63
1,2,Sao Joao Da Barra,BR,78.89,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
2,6,Labuhan,ID,75.29,broken clouds,-6.8844,112.2051,DoubleFAG
3,12,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
4,19,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Hambantota,LK,79.20,overcast clouds,6.1241,81.1185,Bungalow 63
1,2,Sao Joao Da Barra,BR,78.89,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
2,6,Labuhan,ID,75.29,broken clouds,-6.8844,112.2051,DoubleFAG
3,12,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
4,19,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,23,Hilo,US,82.27,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
6,24,Bathsheba,BB,86.61,few clouds,13.2167,-59.5167,Atlantis Hotel
7,29,Oum Hadjer,TD,76.23,light rain,13.2954,19.6966,NaN
8,40,Majene,ID,77.47,broken clouds,-3.5403,118.9707,Kost Putri RABBANI
9,41,Faanui,PF,77.90,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 238
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              239 non-null    int64  
 1   City                 239 non-null    object 
 2   Country              239 non-null    object 
 3   Max Temp             239 non-null    float64
 4   Current Description  239 non-null    object 
 5   Lat                  239 non-null    float64
 6   Lng                  239 non-null    float64
 7   Hotel Name           225 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 16.8+ KB


In [6]:
# 4b. Determine of their are empty rows
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 238
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              239 non-null    int64  
 1   City                 239 non-null    object 
 2   Country              239 non-null    object 
 3   Max Temp             239 non-null    float64
 4   Current Description  239 non-null    object 
 5   Lat                  239 non-null    float64
 6   Lng                  239 non-null    float64
 7   Hotel Name           225 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 16.8+ KB


In [7]:
#4b.Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hambantota,LK,79.20,overcast clouds,6.1241,81.1185,
1,Sao Joao Da Barra,BR,78.89,clear sky,-21.6403,-41.0511,
2,Labuhan,ID,75.29,broken clouds,-6.8844,112.2051,
3,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,
4,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,
5,Hilo,US,82.27,overcast clouds,19.7297,-155.0900,
6,Bathsheba,BB,86.61,few clouds,13.2167,-59.5167,
7,Oum Hadjer,TD,76.23,light rain,13.2954,19.6966,
8,Majene,ID,77.47,broken clouds,-3.5403,118.9707,
9,Faanui,PF,77.90,light rain,-16.4833,-151.7500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the Latitude and Longtitude to Location Key for the Params Dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hambantota,LK,79.20,overcast clouds,6.1241,81.1185,Bungalow 63
1,Sao Joao Da Barra,BR,78.89,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
2,Labuhan,ID,75.29,broken clouds,-6.8844,112.2051,DoubleFAG
3,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
4,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
234,Cabatuan,PH,76.12,moderate rain,10.8794,122.4860,MVC Transient Room
235,Navoi,UZ,87.28,clear sky,40.0844,65.3792,Silk Road Hotel Termez
236,Vaitape,PF,77.90,light rain,-16.5167,-151.7500,Conrad Bora Bora Nui
237,Oyama,JP,84.58,broken clouds,36.3000,139.8000,Toyoko Inn Oyama-eki Higashi-guchi No.1


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hambantota,LK,79.20,overcast clouds,6.1241,81.1185,Bungalow 63
1,Sao Joao Da Barra,BR,78.89,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
2,Labuhan,ID,75.29,broken clouds,-6.8844,112.2051,DoubleFAG
3,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
4,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
234,Cabatuan,PH,76.12,moderate rain,10.8794,122.4860,MVC Transient Room
235,Navoi,UZ,87.28,clear sky,40.0844,65.3792,Silk Road Hotel Termez
236,Vaitape,PF,77.90,light rain,-16.5167,-151.7500,Conrad Bora Bora Nui
237,Oyama,JP,84.58,broken clouds,36.3000,139.8000,Toyoko Inn Oyama-eki Higashi-guchi No.1


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Hotel Name}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
locations

,Lat,Lng
0,6.1241,81.1185
1,-21.6403,-41.0511
2,-6.8844,112.2051
3,-20.3297,-40.2925
4,22.0752,-159.3190
...,...,...
234,10.8794,122.4860
235,40.0844,65.3792
236,-16.5167,-151.7500
237,36.3000,139.8000


In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))